In [1]:
import os
import glob
import polars as pl
import pandas as pd
import plotly.express as px

from utils import hermes_download_client
from utils import ambient_parameter_conversion as apc
from utils import calibration_processing as cp

DATA_DIRECTORY = os.environ.get("DATA_DIRECTORY")
PICARRO_DATA_DIRECTORY = os.environ.get("PICARRO_DATA_DIRECTORY")

sensor_id = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

# customize pipeline
download_files = True
merge_picarro_files = False

# Download to local db

In [2]:
# download from hermes database
# Use Download/download_from_hermes notebook
if download_files:
    
    while(True):
        try:
            component = hermes_download_client.Extract()
            result = component.execute()
            break
        except Exception as e:
            print(e)

Start downloading from datetime: 
did not find a measurements-chunk-*.parquet
reading last creation_timestamp from acropolis.parquet
2024-07-04 10:37:25.410000+00:00
Performing merge.
Deleting merged chunks.


# Process Picarro Data

In [ ]:
if merge_picarro_files:
    filenames = glob.glob(PICARRO_DATA_DIRECTORY + "/*/*/*.dat")

    # read all *.dat picarro measurement files and add to single db
    df_list = []
    for filename in filenames:
        df_list.append(pd.read_csv(filename,sep='\s+'))

    df_p_files = pd.concat(df_list, ignore_index=True)
    df_p_files["datetime"] = pd.to_datetime((df_p_files['DATE'] + ' ' + df_p_files['TIME']))
    df_p_files.sort_values(by='datetime', inplace = True)

    df_p_files.to_parquet(path = os.path.join(DATA_DIRECTORY, "input", "picarro.parquet"))

    #Calibration

    # TODO: Add ability for multiple calibration dates
    # before 23.10
    # picarro_slope = 1.0061589132696314
    # picarro_intercept = 0.14607153970888476

    # after 23.10
    #picarro_slope = 1.0063874771746113
    #picarro_intercept = 0.06621464961165202
    
     
    #after 18.12
    picarro_slope = 1.0060713120261249
    picarro_intercept = 0.08088569875155827
    
    #1m averaged corrected Picarro dataset
    df_p_1m = pl.scan_parquet(os.path.join(DATA_DIRECTORY,"input", "picarro.parquet")) \
        .with_columns(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp")) \
        .sort("creation_timestamp") \
        .with_columns((pl.col("CO2_dry") * picarro_slope + picarro_intercept).alias("CO2_corr")) \
        .group_by_dynamic("creation_timestamp", every='1m') \
        .agg(pl.all().exclude("creation_timestamp").mean()).collect() \
        .select(["creation_timestamp", "CO2_corr", "h2o_reported"]) \
        .with_columns([pl.lit(picarro_slope).alias("slope"),
            pl.lit(picarro_intercept).alias("intercept"),
            pl.lit("Picarro").alias("system_name"),
            pl.lit(0.0).alias("diff")]
            ) \

    df_p_1m.write_parquet(os.path.join(DATA_DIRECTORY,"processed", "1m_cal_corr_picarro.parquet"))
    
    
    #1h averaged corrected Picarro dataset
    df_p_10m = df_p_1m.sort("creation_timestamp") \
        .group_by_dynamic("creation_timestamp", every='10m') \
        .agg(pl.all().exclude("creation_timestamp").mean())
        
    df_p_10m.write_parquet(os.path.join(DATA_DIRECTORY,"processed", "10m_cal_corr_picarro.parquet"))
    
    #1h averaged corrected Picarro dataset
    df_p_1h = df_p_1m.sort("creation_timestamp") \
        .group_by_dynamic("creation_timestamp", every='1h') \
        .agg(pl.all().exclude("creation_timestamp").mean())
        
    df_p_1h.write_parquet(os.path.join(DATA_DIRECTORY,"processed", "1h_cal_corr_picarro.parquet"))

# Import data files

In [3]:
# load calibration bottle concentrations (preprocessed)
df_gas = pl.read_csv(os.path.join(DATA_DIRECTORY,"input", "averaged_gases.csv"))
# load local db: acropolis.parquet
df_raw = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "download", "acropolis.parquet")) \
    .filter(pl.col("system_name") != "test-sensor") 
    
df_p_1m = pl.read_parquet(os.path.join(DATA_DIRECTORY,"processed", "1m_cal_corr_picarro.parquet"))  

# Perform Dry-Wet Conversion

### Measurement Data

In [4]:
# aggregate data to 1m
df_wet_measurements = df_raw.sort("creation_timestamp") \
    .filter(pl.col('gmp343_filtered') > 0) \
    .filter(pl.col('gmp343_temperature') > 0) \
    .filter(pl.col('sht45_humidity') > 0) \
    .filter(pl.col('bme280_pressure') > 0) \
    .group_by_dynamic("creation_timestamp", every='1m', by= "system_name")  \
    .agg(pl.all().exclude(["creation_timestamp","system_name"]).mean()) \
    
# perform dry conversion for measurement data                
df_dry_measurements = df_wet_measurements.with_columns(pl.struct(['gmp343_temperature','sht45_humidity']) \
    .map_elements(lambda x: apc.rh_to_ah(x['sht45_humidity'],apc.absolute_temperature(x['gmp343_temperature'])))
    .alias("h2o_ah")) \
    .with_columns(pl.struct(['gmp343_temperature','sht45_humidity','bme280_pressure'])
    .map_elements(lambda x: (apc.rh_to_molar_mixing(x['sht45_humidity'],apc.absolute_temperature(x['gmp343_temperature']),x['bme280_pressure']*100))*100) \
    .alias("h2o_v%")) \
    .with_columns(pl.struct(['gmp343_filtered','gmp343_temperature','sht45_humidity','bme280_pressure']) \
    .map_elements(lambda x: apc.calculate_co2dry(x['gmp343_filtered'],x['gmp343_temperature'],x['sht45_humidity'],x['bme280_pressure']*100))
    .alias("gmp343_dry")) \
    .collect() 

In [5]:
df_dry_measurements.tail(3).select("creation_timestamp","system_name","gmp343_filtered", "h2o_ah", "h2o_v%" ,"gmp343_dry")

creation_timestamp,system_name,gmp343_filtered,h2o_ah,h2o_v%,gmp343_dry
"datetime[μs, UTC]",str,f64,f64,f64,f64
2024-07-02 13:27:00 UTC,"""tum-esm-midcos…",424.842857,11.975675,1.804463,432.649863
2024-07-02 13:28:00 UTC,"""tum-esm-midcos…",424.9,12.068267,1.820836,432.780217
2024-07-02 13:29:00 UTC,"""tum-esm-midcos…",422.7,12.12654,1.834958,430.601353


In [6]:
#df_dry_measurements.write_parquet(os.path.join(DATA_DIRECTORY, "processed", "1m_acropolis_dry.parquet"))

### Calibration Data

In [7]:
df_dry_calibration = df_raw.filter(pl.col("cal_gmp343_filtered") > 0) \
    .filter(pl.col("cal_gmp343_temperature") > 0) \
    .filter(pl.col("cal_bme280_pressure") > 0) \
    .with_columns(pl.col("cal_sht45_humidity").fill_null(0.0)) \
    .with_columns(pl.struct(['cal_gmp343_temperature','cal_sht45_humidity','cal_bme280_pressure'])
    .map_elements(lambda x: apc.rh_to_molar_mixing(x['cal_sht45_humidity'],apc.absolute_temperature(x['cal_gmp343_temperature']),x['cal_bme280_pressure']*100)) \
    .alias("cal_h2o_v%")) \
    .with_columns(pl.struct(['cal_gmp343_filtered','cal_gmp343_temperature','cal_sht45_humidity','cal_bme280_pressure']) \
    .map_elements(lambda x: apc.calculate_co2dry(x['cal_gmp343_filtered'],x['cal_gmp343_temperature'],x['cal_sht45_humidity'],x['cal_bme280_pressure']*100))
    .alias("cal_gmp343_dry")) \
    .collect()

In [8]:
df_dry_calibration.tail(3).select("creation_timestamp","system_name","cal_gmp343_filtered", "cal_h2o_v%", "cal_gmp343_dry")

creation_timestamp,system_name,cal_gmp343_filtered,cal_h2o_v%,cal_gmp343_dry
"datetime[μs, UTC]",str,f64,f64,f64
2024-06-29 03:31:49.980 UTC,"""tum-esm-midcos…",561.0,0.000989,561.55527
2024-06-29 03:32:00.050 UTC,"""tum-esm-midcos…",563.9,0.000996,564.462419
2024-06-29 03:32:10 UTC,"""tum-esm-midcos…",567.6,0.00099,568.162406


# Calculate Calibration Correction

In [9]:
df_slope_intercept = df_dry_calibration.filter((pl.col("cal_bottle_id") > 0) & (pl.col("cal_bottle_id") < 81)) \
    .join(df_gas.cast({"cal_bottle_id": pl.Float64}), on = ["cal_bottle_id"], how= "left") \
    .with_columns(diff = pl.col("cal_gmp343_dry") - pl.col("cal_bottle_CO2")) \
    .with_columns((pl.col("creation_timestamp").dt.date()).alias("date")) \
    .sort("date") \
    .group_by([pl.col("date"), pl.col("system_name"), pl.col("cal_bottle_id")]) \
    .agg([
        pl.col("cal_sht45_humidity"),
        pl.col("cal_gmp343_temperature"),
        pl.col("cal_bme280_pressure"),
        pl.col("cal_gmp343_filtered"),
        pl.col("cal_gmp343_dry"),
        pl.col("cal_bottle_CO2").last(),
        pl.col("creation_timestamp").last(),
        ]) \
    .with_columns([
        (pl.col("cal_sht45_humidity").list.len()).alias("len"),
        pl.when(pl.col("cal_bottle_CO2") > 460).then(pl.lit("high")).otherwise(pl.lit("low")).alias("cal_bottle_span"),
        pl.col("cal_gmp343_dry").map_elements(lambda x: cp.process_bottle(x)).alias("median_cal_gmp343_dry"),
        pl.col("cal_gmp343_temperature").map_elements(lambda x: cp.process_bottle(x)).alias("median_cal_gmp343_temperature"),
        pl.col("cal_bme280_pressure").map_elements(lambda x: cp.process_bottle(x)).alias("median_cal_bme280_pressure"),
        pl.col("cal_sht45_humidity").map_elements(lambda x: cp.process_bottle(x)).alias("median_cal_sht45_humidity")
        ]) \
    .with_columns([
        (pl.col("median_cal_gmp343_dry") - pl.col("cal_bottle_CO2")).alias("diff_cut_avg_gmp343_dry")
                  ]) \
    .filter(pl.col("median_cal_gmp343_dry") > 0) \
    .sort("cal_bottle_span") \
    .group_by(["date", "system_name"]) \
    .agg([
        pl.col("cal_bottle_span"),
        pl.col("median_cal_gmp343_dry"),
        pl.col("cal_bottle_CO2"),
        pl.col("creation_timestamp").last()
        ]) \
    .with_columns(pl.struct(['median_cal_gmp343_dry','cal_bottle_CO2']) \
    .map_elements(lambda x: cp.two_point_calibration(x['median_cal_gmp343_dry'],x['cal_bottle_CO2'])) \
    .alias('slope, intercept')) \
    .with_columns([(pl.col("slope, intercept").list.first()).alias("slope"),
                   (pl.col("slope, intercept").list.last()).alias("intercept"),
                   (pl.col("median_cal_gmp343_dry").list.first()).alias("median_bottle_1"),
                   (pl.col("median_cal_gmp343_dry").list.last()).alias("median_bottle_2"),
                   ]) \
    .select("creation_timestamp", "date", "system_name", "slope", "intercept","median_bottle_1", "median_bottle_2") \
    .filter(pl.col("slope") > 0) \
    .with_columns(pl.col("system_name").str.extract(r'(\d+)',1).str.to_integer().alias("system_id"))

In [10]:
df_slope_intercept.head(3)

creation_timestamp,date,system_name,slope,intercept,median_bottle_1,median_bottle_2,system_id
"datetime[μs, UTC]",date,str,f64,f64,f64,f64,i64
2023-06-30 02:31:47.490 UTC,2023-06-30,"""tum-esm-midcos…",1.016737,-12.998247,815.6,406.859797,8
2023-07-04 06:20:48.710 UTC,2023-07-04,"""tum-esm-midcos…",1.02365,1.054793,788.6,396.25,5
2023-07-04 06:38:24.110 UTC,2023-07-04,"""tum-esm-midcos…",1.046599,-8.136548,789.348013,389.51608,2


In [11]:
# safe results to parquet
df_slope_intercept.write_parquet(os.path.join(DATA_DIRECTORY, "processed", "slope_intercept_acropolis.parquet"))

In [12]:
fig = px.scatter(df_slope_intercept.filter((pl.col("slope") > 0.7) & (pl.col("slope") < 1.1)), x="creation_timestamp", y = "slope", color = "system_name")
fig.show()
fig = px.histogram(df_slope_intercept.filter((pl.col("slope") > 0.7) & (pl.col("slope") < 1.1)), x="slope", color = "system_name")
fig.show()
fig = px.histogram(df_slope_intercept.filter((pl.col("intercept") < 100) & (pl.col("intercept") > -100)), x="intercept", color = "system_name")
fig.show()

# Perform Calibration Correction

## 1m aggregated data

In [13]:
df_dry_measurements = df_dry_measurements.with_columns(pl.col("system_name").str.extract(r'(\d+)',1) \
    .str.to_integer().alias("system_id")) \
    .with_columns(pl.struct(["system_id"]) \
    .map_elements(lambda x: f"mid-cost-{x['system_id']}") \
    .alias("sys_name_short"))

In [14]:
df_systems = []

for id in sensor_id:
    df_system_calibration = df_slope_intercept.filter(pl.col("system_id") == id) \
        .sort("creation_timestamp") \
        .drop("system_id", "system_name", "date")
    
    df_system = df_dry_measurements.filter(pl.col("system_id") == id) \
        .sort("creation_timestamp") \
        .join_asof(df_system_calibration, on="creation_timestamp", strategy="nearest", tolerance="10m") \
        .join(df_p_1m.select("creation_timestamp", "CO2_corr"), on = ["creation_timestamp"], how= "left") \
        .with_columns([
            pl.col("slope").interpolate(),
            pl.col("intercept").interpolate()
            ]) \
        .with_columns([
            pl.col("slope").forward_fill(),
            pl.col("intercept").forward_fill()
            ]) \
        .with_columns(((pl.col("gmp343_dry")) * pl.col("slope") + pl.col("intercept")).alias("gmp343_corrected")) \
        .with_columns((pl.col("CO2_corr") - pl.col("gmp343_corrected")).alias("diff")) \
        .with_columns((pl.col("creation_timestamp").dt.date()).alias("date"))
    
    df_systems.append(df_system)
        

df_dry_corrected_1m = pl.concat(df_systems, how="diagonal")

In [15]:
df_dry_corrected_1m.write_parquet(os.path.join(DATA_DIRECTORY, "processed", "1m_cal_corr_acropolis.parquet"))

## 10m aggregated data

In [16]:
df_dry_corrected_10m = df_dry_corrected_1m.sort("creation_timestamp") \
        .group_by_dynamic("creation_timestamp", every='10m', by=["system_name", "sys_name_short"]) \
        .agg(pl.all().exclude(["creation_timestamp","system_name","sys_name_short"]).mean())
        
df_dry_corrected_10m.write_parquet(os.path.join(DATA_DIRECTORY, "processed", "10m_cal_corr_acropolis.parquet"))

## 1h aggregated data

In [17]:
# TODO: implement despiking algorithm

In [18]:
df_dry_corrected_1h = df_dry_corrected_1m.sort("creation_timestamp") \
        .group_by_dynamic("creation_timestamp", every='1h', by=["system_name", "sys_name_short"]) \
        .agg(pl.all().exclude(["creation_timestamp","system_name","sys_name_short"]).mean())
        
df_dry_corrected_1h.write_parquet(os.path.join(DATA_DIRECTORY, "processed", "1h_cal_corr_acropolis.parquet"))

# Plot data

In [19]:
df_plot_1 = df_p_1m[["creation_timestamp", "system_name", "CO2_corr"]].rename({"CO2_corr": "CO2"})

In [20]:
df_plot_2 = df_dry_corrected_1m[["creation_timestamp", "system_name", "gmp343_corrected"]].rename({"gmp343_corrected": "CO2"})

In [21]:
df_plot = pl.concat([df_plot_1,df_plot_2])

In [22]:
# Quality Check

from datetime import datetime, timezone

start = datetime(2024, 3, 16, 10, 0, 0).replace(tzinfo=timezone.utc)
end = datetime(2024, 3, 16, 22, 0, 0).replace(tzinfo=timezone.utc)

df_temp = df_plot.filter(pl.col("creation_timestamp").is_between(start, end)).filter(pl.col("system_name").is_in(["tum-esm-midcost-raspi-7", "Picarro"]))

fig = px.scatter(df_temp, x="creation_timestamp", y = "CO2", color = "system_name")
fig.show()